In [12]:
import numpy as np
import pickle
import gzip
import cca_core
from CKA import linear_CKA, kernel_CKA

import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from utils import factory
from utils.data_manager import DataManager
from torch.utils.data import DataLoader

import json
import argparse
from trainer import train


### Cargamos los argumentos

In [13]:

def main(dataset="cifar10", model_name="finetune", convnet_type="resnet32",
         second_task_freeze_stage=0, logfilename="experiment_freeze"):
    # Simulando la obtención de argumentos desde argparse
    class Args:
        pass
    
    args = Args()
    args.dataset = dataset
    args.model_name = model_name
    args.convnet_type = convnet_type
    args.second_task_freeze_stage = second_task_freeze_stage
    args.logfilename = logfilename
    args.config = "./exps/finetune.json"
    
    param = load_json(args.config)
    args = vars(args)  # Convirtiendo los argumentos a un diccionario.
    param.update(args)
    return param  # Puedes hacer lo que necesites con los parámetros aquí

def load_json(settings_path):
    with open(settings_path) as data_file:
        param = json.load(data_file)
    return param

# Ejecución del programa con valores predeterminados
args = main()

print(args)

{'prefix': 'reproduce', 'dataset': 'cifar10', 'memory_size': 2000, 'memory_per_class': 20, 'fixed_memory': False, 'shuffle': True, 'init_cls': 5, 'increment': 5, 'model_name': 'finetune', 'convnet_type': 'resnet32', 'device': ['0'], 'seed': [1993], 'second_task_freeze_stage': 0, 'logfilename': 'experiment_freeze', 'config': './exps/finetune.json'}


### Cargemos los modelos

In [14]:
model_task_0 = torch.load('logs/finetune/experiment_freeze_stage/w0_stage_0.pth')

model_task_1 = torch.load('logs/finetune/experiment_freeze_stage/w1_stage_0.pth')

In [15]:
del model_task_0['fc.weight']; del model_task_0['fc.bias']
del model_task_1['fc.weight']; del model_task_1['fc.bias']

### Cargamos los parametros del modelos en la red

In [16]:
model_1 = factory.get_model(args["model_name"], args)
model_2 = factory.get_model(args["model_name"], args)


In [17]:
model_1._network.load_state_dict(model_task_0)
model_2._network.load_state_dict(model_task_1)

<All keys matched successfully>

### Cargamos el entrenamiento

In [18]:
# self._total_classes

data_manager = DataManager(
    args["dataset"],
    args["shuffle"],
    args["seed"],
    args["init_cls"],
    args["increment"],
    args["second_task_freeze_stage"]
)

test_dataset = data_manager.get_dataset(
    np.arange(0, 10), source="test", mode="test"
)

test_loader = DataLoader(
    test_dataset, batch_size=10_000, shuffle=False, num_workers=8
)

Files already downloaded and verified
Files already downloaded and verified


### Extractor from stage

In [19]:
for i, (_, inputs, targets) in enumerate(test_loader):
    #inputs, targets = inputs.to('cuda'), targets.to('cuda')
    m1_vectors_stage = model_1._network.extract_vector_stage(inputs)
    m2_vectors_stage = model_2._network.extract_vector_stage(inputs)
    print(i)


0


In [20]:
m1_vectors_stage[2].shape

torch.Size([10000, 64, 8, 8])

In [21]:
# 'fmaps': [x_1, x_2, x_3],
sims: dict = {}
save: list = [[], []]

for i in range(3):
    x_1 = m1_vectors_stage[i]
    x_2 = m2_vectors_stage[i]

 
    avg_x1 = x_1.mean(dim=(2, 3)).detach().numpy()  
    avg_x2 = x_2.mean(dim=(2, 3)).detach().numpy()
    linear_CKA_  = linear_CKA(avg_x1, avg_x2)
    kernel_CKA_ = kernel_CKA(avg_x1, avg_x2)
    print('Linear CKA: {}'.format(linear_CKA_))
    print('RBF Kernel CKA: {}'.format(kernel_CKA_))

    save[0].apppend(linear_CKA_)
    save[1].apppend(kernel_CKA_)

sims["linear_CKA"] = save[0]
sims["kernel_CKA"] = save[1]




Linear CKA: 0.8057130688829804
RBF Kernel CKA: 0.8479895821510914
Linear CKA: 0.7855459658871228
RBF Kernel CKA: 0.793218121828177
Linear CKA: 0.3665605739498378
RBF Kernel CKA: 0.3906476069310203


In [22]:
# Especifica la ruta donde deseas guardar el archivo JSON
ruta = '/home/emendezc/investigacion/dinov2-knn/PyCIL/logs/finetune/experiment_freeze_stage/sims.json'

# Guarda el diccionario como un archivo JSON en la ruta especificada
with open(ruta, 'w') as archivo:
    json.dump(sims, archivo)

print("Archivo JSON guardado con éxito.")

Archivo JSON guardado con éxito.


In [23]:
# CCA
a_results = cca_core.get_cca_similarity(avg_x1.T, avg_x2.T, epsilon=1e-10, verbose=False)
print("Mean CCA similarity", np.mean(a_results["cca_coef1"]))

Mean CCA similarity 0.36354295945478216
